In [128]:
import pandas as pd
import numpy as np
import fosforml
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

In [129]:
import seaborn as sns
import matplotlib.pyplot as plt 
plt.style.use('fivethirtyeight')
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [130]:
my_session.connection.database

'ASSORTMENT_PLANNING'

In [131]:
my_session.connection.schema

'CPG_BRONZE'

In [132]:
table_name = "ASSORTMENT_PLANNING.CPG_BRONZE.SALES_CLEAN"

In [133]:
data = my_session.sql("select * from {}".format(table_name))

In [134]:
type(data)

snowflake.snowpark.dataframe.DataFrame

In [135]:
df=sf_df.to_pandas()

In [136]:
type(df)

pandas.core.frame.DataFrame

In [137]:
df.head(5)

,MNTH_CODE,TRANS_DATE,START_DATE,SALES_VALUE,SALES_UNITS,SALES_VOLUME,SALES_PTR_VALUE,OC_CODE,DISTRIBUTOR_CODE,OUTLET_CODE,CITY,STATE,COUNTY,STREET,PRODUCT_CODE,CATEGORY,SUBCATEGORY,BRAND
0,202401,2024-01-18,2024-01-02,1537.89,16,0.001712,1585.454545,202201,DB0706,OL144111,Sedalia,Missouri,City Center,Str4,PRD0064,Hair Care,Conditioner,Bamboo
1,202401,2024-01-28,2024-01-02,518.18,3,0.001125,518.181818,202201,DB0706,OL238706,Williston,North Dakota,City Center,Str1,PRD0113,Perfume and Deodrants,Unisex Deodrant,Navy Blue
2,202401,2024-01-13,2024-01-02,113.64,1,0.000250,113.636364,202201,DB0706,OL223076,Silver City,New Mexico,Santa Cruz,Str1,PRD0107,Lotion,Body Lotion,Coral
3,202401,2024-01-06,2024-01-02,285.71,32,0.000832,285.714286,202201,DB0652,OL175529,Dunkirk,New York,Dolphin,Str3,PRD0147,Kids Care,Baby Cream,Mint
4,202401,2024-01-18,2024-01-02,290.91,1,0.000500,290.909091,202201,DB0652,OL48851,Stroudsburg,Pennsylvania,City Center,Str2,PRD0004,Lotion,Body Lotion,Tan


In [138]:
df['COUNTRY'].astype('CATEGORY').cat.codes

KeyError: 'COUNTRY'

In [ ]:
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df[['STATE', 'SALES_UNITS', 'SALES_VOLUME']])

In [ ]:
kmeans = KMeans(n_clusters=2, random_state=0).fit(df_scaled)
df['cluster'] = kmeans.labels_

In [ ]:
print(df)

In [ ]:
def suggest_order_quantity(cluster_data, product_sales):
    avg_sales = cluster_data[product_sales].mean()
    return avg_sales * 1.1  # Adding 10% buffer

In [ ]:
cluster_data = df[df['cluster'] == 0]  # Select a cluster
suggested_qty = suggest_order_quantity(cluster_data, 'sales_volume')
print("Suggested Order Quantity:", suggested_qty)